# About This Notebook
* This notebook evaluates the number of tokens generated by the model EleutherAI/gpt-neo-1.3B with The Pile data
* This is the unshuffled version of the notebook.
* The Evaluation is done by the following methodology

* Select documents within THE PILE 00 dataset with more than or equal to 40 tokens, replace the skipped logits with np.nan
* Utilize the first 20 tokens of each document as input to the model and use the subsequent 20 tokens to calculate the number of correctly predicted tokens
* Correctly predicted tokens are found by comparing the tokens in range 20..40 of the input document with the tokens in range 20--40 of generated document
* This data is stored in the array acc and saved in the file 'withoutshuffle.pkl'
* The following file contains information of the score of the model based on first 10,000 documents of THE PILE 
* The input to this dataset was a [kaggle dataset](https://www.kaggle.com/usaiprashanth/the-pile-train-00) dataset with output in the following [kaggle dataset](https://www.kaggle.com/usaiprashanth/gptmodel-outputs)
* The following notebook is from a [kaggle notebook](https://www.kaggle.com/usaiprashanth/gpt-1-3b-model?scriptVersionId=72761073)

In [ ]:
!pip install lm-dataformat #Library for easy retrieval of The pile data from jsonl.zstd files
!pip install transformers #Library for easy interaction with gpt-neo-1.3B model

     |████████████████████████████████| 2.2 MB 4.4 MB/s 


* Creating the @param model using Huggingface api

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

* LM_Dataformat allows for retrieval of data by providing an iterator method "stream_data"

In [ ]:
import lm_dataformat
pile = lm_dataformat.Reader('../input/the-pile-train-00-dataset/the-eye.eu/public/AI/pile/train/00.jsonl.zst')

* Creating the array @param docs, which contains the first 10,000 documents of THE PILE dataset

In [ ]:
docsiter = iter(pile.stream_data())
docs = []
from tqdm import tqdm 
with tqdm(total=10000, position=0, leave=True) as pbar:
    for i in range(10000):
        docs.append(next(docsiter))
        pbar.update()

100%|██████████| 10000/10000 [00:01<00:00, 7545.18it/s]


In [ ]:
documentidx = list(range(10000))

In [ ]:
import numpy as np
import torch
acc = [[],[],[]]
from sklearn.metrics import accuracy_score
with tqdm(total=10000, position=0, leave=True) as pbar:
    for idx in range(10000):
        doc = docs[documentidx[idx]]
        input_ids = tokenizer(doc, return_tensors="pt").input_ids
        
        if(input_ids.shape[1] < 40):
            acc[0].append(documentidx[idx])
            acc[1].append(input_ids.shape[1])
            acc[2].append(np.nan) #Input tokens are less than 40, append nan as logit
            continue
        
        length = 20 #Length of subsequent tokens, the tokens utilized to calculate number of correctly predicted tokens
        
        gen_tokens = model.generate(input_ids[:,:20].cuda(), do_sample=True, temperature=0.1,pad_token_id=50256,eos_token_id=50256,
                                    min_length=20+length,max_length=40+length).cpu()[0] #Generating tokens from the model
        
        score = torch.sum(input_ids[0,20:length+20] == gen_tokens[20:length+20]).item() #Calculating the number of correctly predicted tokens 
        
        
        #Appending the document index, length of input tokens and the score to the logit array, @param acc
        acc[0].append(documentidx[idx])
        acc[1].append(input_ids.shape[1])
        acc[2].append(score)
        
        idx+=1 #Temporary index, keeps count of current iteration
        pbar.update()

 99%|█████████▉| 9875/10000 [3:16:47<02:29,  1.20s/it]


In [ ]:
import joblib
joblib.dump(acc,'withoutshuffle.pkl') #Saving @param acc

['withoutshuffle.pkl']